In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.exceptions import SpotifyException
import matplotlib.pyplot as plt
import random

print("All libraries imported successfully.")

All libraries imported successfully.


In [2]:
# Load the .env file variables
load_dotenv("../.env")

True

In [ ]:
client_id = os.getenv("SPOTIFY_CLIENT_ID")
client_secret = os.environ.get("SPOTIFY_CLIENT_SECRET")

In [4]:
# Authenticate with Spotify
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [7]:
class SpotifyRecommender:
    def __init__(self, sp):
        self.sp = sp

    def get_available_genres(self):
        """Retrieve available genre seeds from Spotify."""
        return self.sp.recommendation_genre_seeds()['genres']

    def search_artist_id(self, artist_name):
        """Search for an artist and return their Spotify ID."""
        result = self.sp.search(q='artist:' + artist_name, type='artist', limit=1)
        if result['artists']['items']:
            return result['artists']['items'][0]['id']
        print(f"Warning: No artist found for '{artist_name}'")
        return None

    def search_track_id(self, track_name):
        """Search for a track and return its Spotify ID."""
        result = self.sp.search(q='track:' + track_name, type='track', limit=1)
        if result['tracks']['items']:
            return result['tracks']['items'][0]['id']
        print(f"Warning: No track found for '{track_name}'")
        return None
    # Main method for getting recommendations
    def get_track_recommendations(self, track_seeds=None, artist_seeds=None, genre_seeds=None, 
                                  amount=20, country="US", mood=None):
        if not (track_seeds or artist_seeds or genre_seeds):
            raise ValueError("At least one of seed_tracks, seed_artists, or seed_genres must be provided.")
        
        # Convert genre seeds to lowercase and filter based on available genres
        available_genres = self.get_available_genres()
        genre_seeds = [genre.lower() for genre in genre_seeds if genre.lower() in available_genres]
        
        # Set default Spotify audio feature values
        spotify_data = {
            'acousticness': 0.5,
            'danceability': 0.5,
            'duration_ms': 210000,
            'energy': 0.5,
            'instrumentalness': 0.0,
            'speechiness': 0.5,
            'tempo': 120.0,
            'valence': 0.5
        }
        
        # Define mood parameters
        mood_map = {
            'happy': {'valence': 0.8, 'danceability': 0.7},
            'sad': {'valence': 0.2, 'acousticness': 0.7},
            'dance': {'danceability': 0.9, 'energy': 0.8},
            'relaxed': {'valence': 0.6, 'acousticness': 0.8, 'energy': 0.3, 'tempo': 80.0},
            'energetic': {'energy': 0.9, 'danceability': 0.8, 'tempo': 140.0},
            'romantic': {'valence': 0.7, 'acousticness': 0.6, 'energy': 0.4},
            'calm': {'valence': 0.4, 'acousticness': 0.9, 'energy': 0.2, 'tempo': 60.0},
            'angry': {'energy': 0.8, 'valence': 0.2, 'danceability': 0.5, 'tempo': 150.0},
            'focus': {'acousticness': 0.6, 'instrumentalness': 0.5, 'tempo': 100.0},
            'uplifting': {'valence': 0.9, 'energy': 0.7, 'danceability': 0.6, 'tempo': 120.0}
        }
        
        # Apply mood-based filtering if valid mood is provided
        if mood and mood.lower() in mood_map:
            spotify_data.update(mood_map[mood.lower()])
        elif mood:
            print(f"Invalid mood '{mood}' provided. Skipping mood-based filtering.")

        # Prepare the input parameters for the Spotify API
        try:
            recommendations = self.sp.recommendations(
                seed_tracks=track_seeds,
                seed_artists=artist_seeds,
                seed_genres=genre_seeds,
                limit=amount,
                country=country,
                target_acousticness=spotify_data.get('acousticness', 0.5),
                target_danceability=spotify_data.get('danceability', 0.5),
                target_duration_ms=int(spotify_data.get('duration_ms', 210000)),
                target_energy=spotify_data.get('energy', 0.5),
                target_instrumentalness=spotify_data.get('instrumentalness', 0.0),
                target_speechiness=spotify_data.get('speechiness', 0.5),
                target_tempo=spotify_data.get('tempo', 120.0),
                target_valence=spotify_data.get('valence', 0.5)
            )
            # Return only track names as per requirements
            return [(track['name'], track['artists'][0]['name']) for track in recommendations['tracks']]
        except SpotifyException as e:
            print(f"Error: {e}")
            return None


In [8]:
def get_user_input():
    """Prompt the user for genres, artists, songs, and mood."""
    # Get available genres
    available_genres = SpotifyRecommender(sp).get_available_genres()
    print("Available genres:", ", ".join(available_genres))
    
    # Genre input
    genres = input("Enter genres separated by commas (or press Enter to skip): ").lower().split(',')
    genres = [genre.strip() for genre in genres if genre.strip() in available_genres]

    # Artist input
    artists = input("Enter artist names separated by commas (or press Enter to skip): ").split(',')
    artists = [artist.strip() for artist in artists if artist.strip()]

    # Song input
    songs = input("Enter song names separated by commas (or press Enter to skip): ").split(',')
    songs = [song.strip() for song in songs if song.strip()]

    # Mood input
    available_moods = ["happy", "sad", "dance", "relaxed", "energetic", "romantic", "calm", "angry", "focus", "uplifting"]
    print("Available moods:", ", ".join(available_moods))
    mood = input("Enter a mood (or press Enter to skip): ").strip().lower()
    if mood not in available_moods and mood:
        print("Invalid mood selected. Skipping mood filter.")
        mood = None

    return genres, artists, songs, mood

In [9]:
# Example usage
def main():
    print("Welcome to the Spotify Recommendation System!")
    
    # Collect genres, artists, songs, and mood from user input
    genres, artists, songs, mood = get_user_input()
    
    # Convert artist and song names to IDs
    recommender = SpotifyRecommender(sp)
    genre_seeds = genres
    artist_seeds = [recommender.search_artist_id(artist) for artist in artists if recommender.search_artist_id(artist)]
    track_seeds = [recommender.search_track_id(song) for song in songs if recommender.search_track_id(song)]

    # Fetch and display recommendations
    recommendations = recommender.get_track_recommendations(track_seeds=track_seeds, 
                                                            artist_seeds=artist_seeds, 
                                                            genre_seeds=genre_seeds, 
                                                            mood=mood)
    if recommendations:
        print("\nRecommended Songs:")
        for idx, (track_name, artist_name) in enumerate(recommendations, start=1):
            print(f"{idx}. {track_name} by {artist_name}")
    else:
        print("No recommendations available. Try using different input options.")

In [11]:
# Run the program
if __name__ == "__main__":
    main()

Welcome to the Spotify Recommendation System!
Available genres: acoustic, afrobeat, alt-rock, alternative, ambient, anime, black-metal, bluegrass, blues, bossanova, brazil, breakbeat, british, cantopop, chicago-house, children, chill, classical, club, comedy, country, dance, dancehall, death-metal, deep-house, detroit-techno, disco, disney, drum-and-bass, dub, dubstep, edm, electro, electronic, emo, folk, forro, french, funk, garage, german, gospel, goth, grindcore, groove, grunge, guitar, happy, hard-rock, hardcore, hardstyle, heavy-metal, hip-hop, holidays, honky-tonk, house, idm, indian, indie, indie-pop, industrial, iranian, j-dance, j-idol, j-pop, j-rock, jazz, k-pop, kids, latin, latino, malay, mandopop, metal, metal-misc, metalcore, minimal-techno, movies, mpb, new-age, new-release, opera, pagode, party, philippines-opm, piano, pop, pop-film, post-dubstep, power-pop, progressive-house, psych-rock, punk, punk-rock, r-n-b, rainy-day, reggae, reggaeton, road-trip, rock, rock-n-roll